<img src="./images/DLI_Header.png" style="width: 400px;">

## Assessment
Congratulations on going through today's course! Hopefully you've learned some valuable skills along the way. Now it's time to put those skills to the test. In this assessment you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of 92% in order to pass the assessment, though we encourage you to try for even better accuracy. You will have the use the skills that you learned in the previous exercises. Specifically we suggest you use some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate, or better, on the test dataset, you will save your model, and then assess its accuracy. Let's get started! 

### The Dataset
In this exercise, you will train a model to recognize fresh and rotten fruits. If you'd like, you can look at the dataset structure in the `fruits` folder. There are six categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with categorical_crossentropy, as we have more than two categories.

### Load Imagenet Base Model
We encourage you to start with a model pretrained on imagenet. You'll need to load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. You can pick a desired height and width for the model to expect, but we recommend 224 by 224 pixels. If you need a reference for setting up the pretrained model, please take a look at notebook 7 where you first implemented transfer learning.

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    # weights='imagenet',  # Load weights pre-trained on ImageNet.
    weights='data/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',  # Load weights pre-trained from local files.
    input_shape=(224, 224, 3),
    include_top=False)

### Freeze Base Model
Next, we suggest you freeze the base model, as you did in notebook 7. This is done so that all the learning from the imagenet dataset does not get destroyed in the initial training.

In [ ]:
# Freeze base model
base_model.trainable = False

### Add Layers to Model
Now it's time to add layers to your pretrained model. You can again use notebook 7 as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [ ]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

# 
x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

### Compile Model
Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [ ]:
model.compile(loss = 'categorical_crossentropy' , metrics = ['accuracy'])

### Augment the Data
If you'd like, try to augment the data to improve the dataset. Feel free to look at notebook 4 and notebook 7 for augmentation examples. You can also look at the documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but you may find it helpful to get to 92% accuracy when you train.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator() # Data augmentation is not required to pass

# datagen = ImageDataGenerator(rotation_range=20, 
#                   zoom_range=0.15, 
#                   width_shift_range=0.2, 
#                   height_shift_range=0.2, 
#                   shear_range=0.15, 
#                   horizontal_flip=True)

### Load Dataset
Now it's time to load the train and test datasets. You'll have to pick the right folders, as well as the right target_size of the images (it needs to match the height and width input of the model you've created). If you'd like a reference, you can check out notebook 7.

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('/dli/task/data/fruits/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical"
                                      )
# load and iterate test dataset
test_it = datagen.flow_from_directory('/dli/task/data/fruits/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical"
                                     )

### Train the Model
Time to train the model! Pass the train and test iterators into the fit function, as well as setting your desired number of epochs.

----------

In [ ]:
train_it.samples

In [ ]:
train_it.batch_size

In [ ]:
test_it.samples

----------

In [ ]:
train_it.samples//train_it.batch_size

In [ ]:
model.fit(train_it,
          validation_data=test_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=10)

### Unfreeze Model for Fine Tuning
If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest you try fine tuning the model with a very low learning rate. You may again use notebook 7 as a reference.

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [ ]:
model.fit(train_it,
          validation_data=test_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=2)

### Evaluate the Model

Hopefully you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, you can evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. You'll want to have an accuracy value of .92 or higher. 

In [ ]:
model.evaluate(test_it, steps=test_it.samples/test_it.batch_size)

### Run the Assessment

To assess your model against the test data, run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your test data iterator is called `test_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [ ]:
from run_assessment import run_assessment

In [ ]:
run_assessment(model, test_it)